# Visualizing SuperQuadrics with Marching Cubes and Marching Tetrahedra
Matthew Lind,
University of Illinois at Urbana-Champaign

In this project, you will implement the Marching Cubes and Marching Tetrahedra algorithms to generate isosurfaces from volumetric data.  The superquadric primitives will be employed as the volumetric data to be visualized to illustrate the differences.  Before proceeding with the project, you should read chapter 2 "Marching Cubes and Variants" from the book __Isosurfaces: Geometry, Toplogy, and Algorithms__ by Rephael Wenger (CRC Press / AK Peters Publishing).  The book can be downloaded for free as a .pdf from the CRC Press website when accessed through the UIUC library.

__Marching Cubes__ is an algorithm for generating isosurfaces from volumetric data by subdividing a volume into virtual cubes and marching through the volume evaluating each cube in turn.  The results of each cube are stitched together to form a contiguous polygon mesh as the isosurface.  Marching Cubes can be easily implemented with the use of a lookup table and performs in linear time making it a practical choice for generating isosurfaces.  However the algorithm has a few weaknesses.  One weakness is data in some cubes may be ambiguous leading to uncertainty as to how the isosurface should be constructed.  Either additional computation, or conscious biased choices in the lookup table must be made to resolve ambiguities and produce a cohesive isosurface.  Reference: [Marching Cubes (Wikipedia)](https://en.wikipedia.org/wiki/Marching_cubes)

__Marching Tetrahedra__ is a variation employed by many who couldn't use Marching Cubes due to patent restrictions.  Marching Tetrahedra can be viewed as an extension of Marching Cubes as it follows the same principle but dissects the cubes into tetrahedra producing more granularity which resolves all ambiguity and produces a correct isosurface every time.  However, this comes at the expense of additional computation and significantly more triangles and vertices in the resulting isosurface.  Within the Marching tetrahedra algorithm are variants decomposing the cube into 5 tetrahedra or 6 tetrahedra.  Each has their strengths and weaknesses as we'll see later.  Reference: [Marching Tetrahedra (Wikipedia)](https://en.wikipedia.org/wiki/Marching_tetrahedra).

__Superquadrics__ are a class of primitives defined by variations of the sphere equation.  In addition to scientific visualization, superquadrics have been employed in computer graphics under the name of 'metaballs' or 'blobby particles' for density based modeling techniques favorable to haptic sculpting and simulating modeling with clay.  Metaballs are usually enhanced with features such as attractive and repulsive blending enabling metaballs to act as carving and shaping tools in the modeling process, and to represent liquids and other formless shapes.  When the number of metaballs becomes large, representation moves to point clouds employing blobby particles.   One example of superquadrics used as metaballs in commercial filmmaking is the main animated character in the 1997 movie [Flubber](https://en.wikipedia.org/wiki/Flubber_(film)).  Commerical software applications using this technique include [Z-Brush (PixelLogic)](https://pixologic.com/) and [MudBox (AutoDesk)](https://www.autodesk.com/products/mudbox/overview).   Reference: [SuperQuadrics (Wikipedia)](https://en.wikipedia.org/wiki/Superquadrics)

In [1]:
import numpy as np
import pyvista as pv
import time

from data import *

### constants ###
X = 0    # convenience for accessing indices in vectors, arrays, ...
Y = 1
Z = 2

U = 0
V = 1
W = 2

emValues = [4, 5, 6, 7, 6, -6, -7, 5, 11, 10, -11, 10, 15, 17, 16, -16, -17, -18, -19]

In [2]:
#=======================================================================
# StopWatch() - Primitive stopwatch for tracing execution times during debugging/testing
#=======================================================================
class StopWatch( object ):
    def __init__( self, n=0, start=0, now=0, splt=0, total=0 ):
        self.start = n
        self.now   = time.perf_counter()
        self.split = splt
        self.total = total

    # === Methods ===
    
    # Get current interval
    def GetSplit( self ):
        self.start = self.now
        self.now   = time.perf_counter()
        self.split = ( self.now - self.start ) # * 0.001
        self.total += self.split
        return( self.split )

    # Get last reported split time
    def GetLastSplit( self ):
        return( self.split )

    # Get total time since object initialized
    def GetTotal( self ):
        return( self.total )

    # Reset counter to specified time - or zero if not specified
    def Reset( self, n=0 ):
        self.total = n

# Part 1: Generate a polygon mesh from data.

In this project isosurfaces are represented as polygon meshes.  Polygon meshes can represent just about any surface due to it's arbitrary nature, and most algorithms can be employed without sophisticated mathematics.  However, polygon meshes are approximations of the surface they represent as all vertices are connected by linear edges. For example, it would require near infinite resolution to represent a highly curved surface as the linear edges would require many vertices to be placed close together to approximate the curvature making for an impractical choice.  Therefore, the result is always a compromise between the desired output and the available computing resources.

Polygon meshes can be created in PyVista using the pyvista.PolyData() class.  PolyData requires two lists:
1. Vertex Positions
2. Polygon connectivity.

Vertex positions are provided as a list of XYZ position vectors described in the coordinate space of the object.  Each position vector is a 1D array of scalars of size 3.

The polygon connectivity is provided as a list of face descriptions.  Each face description a 1D array of __N+1__ entries, where __N__ represents the number of sides in the polygon and is the first value in the face description.  The trailing values are the indices of the vertices in the vertex positions list which form the polygon and specified in the order that respects the _right hand rule_.  The last specified index will automatically be connected by an edge to the first specified index to complete the polygon.  Face descriptions must be provided as row vectors, hence the use of np.hstack() below to format the list.  Keep that in mind if using something other than numpy to form your data.

__Right hand rule__:  If you use your right hand and curl the fingers to point in the direction that follows the order the vertices are connected, the thumb will point in the direction of the surface normal.  If defined correctly, the surface normal will point out of your screen when looking at a 2D polygon (the vertices should be connected in counter-clockwise order around the perimeter of the polygon).  If the order is incorrect, the polygon will be inverted, or in some extreme cases, throw errors.  Exact details of how this situation is handled varies by API.

In the cell below, construct two polygon mesh cubes, each from a vertex positions list and polygon connectivity list.  The vertex positions should be confined to the range [0...1] on the X, Y, and Z axes.  For the first polygon mesh, desribe the faces as triangles.  In the second polygon mesh, describe each face as a quadrilateral polygon.

In [3]:
# mesh points
vertices = np.array([
   # your code here
])

# faces - triangle mesh
facesT = np.hstack([  
   # your code here
])

# faces - quadrilateral mesh
facesQ = np.hstack([  
   # your code here
])

aPolygonMeshes = [
    pv.PolyData( vertices, facesT ),
    pv.PolyData( vertices, facesQ )
]

ValueError: need at least one array to concatenate

## Display the polygon mesh

One option to display the resulting polygon mesh is to use the Pyvista.Plotter(). This method is useful when you want to see multiple polygon meshes in a single view, or when you need to decorate the view with intformation.

The general procedure for creating a view with Pyvista.Plotter() is as follows:

- Create a subplot.
- Add elements to the subplot (e.g. a polygon mesh)
- Specify camera parameters.
- (optional) Decorate with text.

Code for a simple plotter has been provided in the cell below.

In [ ]:
titles = ["Cube - Triangles", "Cube - Quadrilaterals"]

plotter = pv.Plotter( shape=(1, 2) )

for i in range(0,len( aPolygonMeshes)):
    
    mesh = aPolygonMeshes[i]
    
    # create a new subplot to occupy part of the final frame.
    plotter.subplot(0,i)
    
    # set camera
    plotter.camera_position = [
        (3,3,2),        # position
        (0.5,0.5,0.5),  # look-at
        (0,0,1)         # up vector
    ]
    
    # apply text
    plotter.add_text( titles[i] +
        '\n  Vertices: \t' + str( len( mesh.points ) ) +
        '\n  Polygons: \t' + str( mesh.n_faces),
        font_size=12,
        font='courier'
    )
    
    # add the mesh with colormap
    plotter.add_mesh( mesh, show_edges=True, scalars=mesh.points[:,Z], stitle='Z coordinate'  )
    
    # add scalar bar
#     plotter.add_scalar_bar( "Z" )

# display the result
plotter.show(  window_size=[1920,720], screenshot="cubes.png" )

 Your cubes should look like this:

![cubes](./images/polygon_mesh_cubes.png)

### Display the polygon mesh

One option to display the resulting polygon mesh is to use PolyData.plot().  This method is useful when you only want to see the polygon mesh in isolation of all other data in the scene.  We'll see another option later for working with multiple objects.

The plot() method takes multiple (optional) arguments:

- __scalars__: defines values used by the colormap to color the surface.  The number must match the number of vertices or cells (polygons).
- __cpos__: defines the position of the camera in world space, looking back towards (0,0,0).
- __show_edges__: True if edges of polygons should be drawn on the displayed polygon mesh.

__NOTE__: The PyVista documentation fails to mention the camera position is not a true position, but rather an orientation vector relative to the world origin produced by normalizing the camera position coordinate to a unit vector.  This can be demonstrated by specifying very large values for select vertex positions in the vertex list above.  The viewer will automatically adjust to fit the polygon mesh into the view, but preserve the viewing angle as specified in the cpos argument by dollying the camera along that vector.

In [ ]:
# Try each method separately to compare.

# method 1 - plot each vertex with a different color
aPolygonMeshes[0].plot( scalars=np.arange( len( aPolygonMeshes[0].points ) ), cpos=[1, -1, 0.5], show_edges=True )
aPolygonMeshes[1].plot( scalars=np.arange( len( aPolygonMeshes[1].points ) ), cpos=[1, -1, 0.5], show_edges=True )

# method 2 (optional) - plot each polygon with a different color
# aPolygonMeshes[0].plot( scalars=np.arange(12), cpos=[1, -1, 0.5], show_edges=True )
# aPolygonMeshes[1].plot( scalars=np.arange(6),  cpos=[1, -1, 0.5], show_edges=True )

# Part 2: Implement SuperQuadric()

For this project we'll visualize a superquadrics primitive.  Superquadrics are a class of primitives defined by a variation of the sphere equation:
## $$(\frac{|x|}{a})^r + (\frac{|y|}{b})^s + (\frac{|z|}{c})^t = R^2$$
where:
- __x,y,z__: position coordinate of a point on the surface.
- __a,b,c__: local axis scale factors of the superquadric primitive.
- __r,s,t__: exponents which determine the shape of the primitive
- __R__: Radius of the superquadric at point [x,y,z].

The primitive is based on an ellipsoid, but can become any number of other shapes by modifying the scale and exponent parameters:

![SuperQuadric shapes](./images/superquadric_shapes.png)


We must modify the equation for our purposes to generate an isosurface by rearranging the equation to solve for the radius __R__, where R will be compared to our isovalue.  For example, if isovalue = 1.0, we would make the following comparison:

- __R < 1__: position is inside the primitive.
- __R = 1__: position is on surface of the primitive.
- __R > 1__: position is outside the primitive.

The position coordinate will be derived from the grid points of our scalar field.  Each point will be tested to see if it is inside or outside the primitive and recorded in the grid point as the scalar value.

In the cell below, implement the superquadric function to read a position coordinate and return the radius at that location.  SuperQuadric() should accept multiple arguments as input:
-  __position__: (3D vector) position in 3D space to be evaluated (default = [0,0,0])
-     __scale__: (3D vector) scaling factors of the primitive's local axes (default = [1,1,1])
- __exponents__: (3D vector) Exponents used in the equation which define the primitive's shape (default = [2,2,2])

The function should return a scalar indicating the radius from the center of the superquadric primitive.

In [ ]:
#===========================================================================
# superQuadric() - given position (x,y,z), determine if it is inside our outside the primitive.
# If less than 1.0, point is inside the surface.
#
# Parameters:
#  position: (3D vector) position in 3D space to be evaluated (default = [0,0,0])
#     scale: (3D vector) scaling factors of the primitive's local axes (default = [1,1,1])
# exponents: (3D vector) Exponents used in the equation which define the primitive's shape (default = [2,2,2])
#
# Returns:
# (scalar): radius of the primitive
#===========================================================================
def superQuadric( position=[0,0,0], scale=[1,1,1], exponents=[2,2,2] ):
    
    # your code here
    
    return( None )

In [ ]:
r = superQuadric( [-1,-1,-1], [4,2,1], [-3,-1,-5]  )
print( "radius: ", r )

In [ ]:
## test cases ##
np.testing.assert_almost_equal( superQuadric( [0,0,0] ), 0.0, err_msg="Failed case 1", verbose=False )
np.testing.assert_almost_equal( superQuadric( [1,1,1] ), 1.7320508075688772, err_msg="Failed case 2", verbose=False )
np.testing.assert_almost_equal( superQuadric( [-5,-6,-777], [1,1,1], [3,3,3] ), 21658.665101986317, err_msg="Failed case 3", verbose=False )
np.testing.assert_almost_equal( superQuadric( [-1,-2,-100], [4,0.5,12], [7,0.25,14] ), 2790816.4723367887, err_msg="Failed case 4", verbose=False )
np.testing.assert_almost_equal( superQuadric( [-5,-6,-777]  ), 777.0392525477719, err_msg="Failed case 5", verbose=False )
np.testing.assert_almost_equal( superQuadric( [0,0,0], [4,2,1], [-3,-1,-5]  ), 0.0, err_msg="Failed case 6", verbose=False )
np.testing.assert_almost_equal( superQuadric( [-1,-1,-1], [4,2,1], [-3,-1,-5]  ), 8.18535277187245, err_msg="Failed case 7", verbose=False )

print( "superQuadric() tests succeeded" )

# Part 3: Define the Volume

Before we can visualize anything, we must define the domain in which the visualization will occur.  For this project we'll define a rectangular volume in cartesian coordinate space with arbitrary dimension, and define resolution by spacing between the grid points.  This volume will be known as a _scalar field_.  The scalar field is a 3 dimensional grid whose points contain an arbitrary scalar value of importance relative to the data we're visualizing.  The space between the points and shaped liked cubes will be referred to as _cells_.  

The Marching Cubes and Marching Tetrahedra algorithms traverse the scalar field cell-by-cell gathering scalars from the grid points and applying them as values on the corners of the cells.  The cell is then evaluated by looking for discrepancy between vertices sharing an edge.  When a discrepancy is found, it means the isosurface intersects that edge, and we need to insert a vertex at the location of the intersection.  After the cell is evaluated, the inserted vertices are connected to form triangles of the isosurface.  The field can be traversed in any order, but we will do it in the X, Y, then Z order (with Z being the vertical axis) to correlate with the lookup tables we'll define.

The generated isosurface will be affected by a few factors:
- __Cell size__: Distance between adjacent vertices along an axis in the scalar field, and uniform across all axes.
- __Dimensions__: Size of scalar field in each axis.  Defined by 2 positions representing the min and max corners of the bounding box.
- __IsoValue__: Scalar value at which the isosurface will be drawn.

Cell size affects the resolution of the isosurface.  Smaller sizes bring scalar field points closer together producing an isosurface with finer details, but also increase computation and memory requirements significantly as more points are needed to cover a field of the same dimensions.

Dimensions of the scalar field affect how much of the isosurface is seen.  If the isosurface extends beyond the dimensions of the scalar field, it will be truncated at the field's boundary.  This can be a useful tactic when you only want to visualize a portion of a larger data set by transforming the data (or the field) so only the area of interest appears in the scalar field.

Isovalue affects the shape of the isosurface relative to the data set.  This is subjective and varies with the data set.

![Scalar Field](./images/scalar_field.png)

## Implement ScalarField and Pole classes

The Pyvista CurviLinear grid or RectiLinear grid classes would normally be used, but they are a bit cumbersome to work with for our purposes, so we'll devise our own: __ScalarField__ and __Pole__.
 
The __ScalarField__ class implements the scalar field as a modified CurviLinear grid.  Points are connected in matrix-like fashion with a position and scalar value recorded at each point in a _pole_ object.  The scalar field needs to know the dimensions of the grid in the X, Y, and Z directions, and populate the grid with a pole at each grid point.

The ScalarField object should be initialized with the number of _cells_ in each of the major axes, actively store the dimensions of the field, and an array or list for the grid point values (poles).  As a minimum, the ScalarField class should include the following methods:

- __\_\_init\_\_( u, v, w )__: intializes the scalar field to the specified dimensions.
- __setVertex( i, j, k, value )__: Records a pole for the grid point at the specified coordinate.
- __getVertex( i, j, k )__: Returns the pole stored at the specified coordinate.

The __Pole__ class defines a pole as a 3D position vector and a scalar value, storing each independently.  At a minimum, it should include the following methods:

- __\_\_init\_\_( x, y, z, value )__: initializes the x, y, z, and value properties to the specified values, or 0.
- __getValue()__: returns the scalar portion of the pole.
- __setValue( value )__: sets the scalar portion of the pole to scalar _value_.
- __getVector()__: returns the vector portion of the pole.

__NOTE__: Additional properties/methods may be needed.

In [ ]:
#=========================================
# ScalarField()
# 
# defines 3D scalar field with values at the grid points.
#=========================================
class ScalarField( object ):
    def __init__( self, u=0, v=0, w=0 ):
        self.u = u
        self.v = v
        self.w = w
        
    #-------------------------------------------------------------------------------------
    # getIndexFromLocation() - given a grid position coordinate, return it's index in the flattened array
    #-------------------------------------------------------------------------------------
    def getIndexFromLocation( self, i, j, k ):
        return( None )

    #-------------------------------------------------------------------------------------
    # getVertexFromIndex() - given a grid point index, return the pole stored at the location
    #-------------------------------------------------------------------------------------
    def getVertexFromIndex( self, index ):
        return( None )

    #-------------------------------------------------------------------------------------
    # getVertex() - returns pole stored at specified coordinate
    #-------------------------------------------------------------------------------------
    def getVertex( self, i, j, k ):
        return( None )

    #-------------------------------------------------------------------------------------
    # setVertex() - Records pole 'p' at specified coordinate (i,j,k)
    #-------------------------------------------------------------------------------------
    def setVertex( self, i, j, k, p ):
        return( None )

#=========================================
# Pole()
#
# associates a 3D vector with a scalar value
#=========================================
class Pole():
    def __init__( self, x=0, y=0, z=0, value=0 ):
        self.x     = x
        self.y     = y
        self.z     = z
        self.value = value

    #-------------------------------------------------------------------------------------
    # getValue() - return value portion of the pole
    #-------------------------------------------------------------------------------------
    def getValue( self ):
        return( None )

    #-------------------------------------------------------------------------------------
    # setValue() - set value portion of the pole
    #-------------------------------------------------------------------------------------
    def setValue( self, value ):
        return( None )

    #-------------------------------------------------------------------------------------
    # getVector() - return the vector portion of the pole
    #-------------------------------------------------------------------------------------
    def getVector( self ):
        return( None )
    

In [ ]:
## test cases - Pole() ##

pole = Pole( 1, 2, 3, 4 )
np.testing.assert_almost_equal( pole.getValue(), 4, err_msg="Failed case 1", verbose=False )
np.testing.assert_almost_equal( pole.getVector(), [1,2,3], err_msg="Failed case 2", verbose=False )
pole.setValue( -3.1415928 )
np.testing.assert_almost_equal( pole.getValue(), -3.1415928, err_msg="Failed case 3", verbose=False )
print( "Pole() tests succeeded")

## test cases - ScalarField() ##

field = ScalarField(4,4,4)
np.testing.assert_almost_equal( field.dimension, [5,5,5], err_msg="Failed case 1", verbose=False )

field = ScalarField(-4,4,4)
np.testing.assert_almost_equal( field.dimension, [1,5,5], err_msg="Failed case 2", verbose=False )

field = ScalarField( 4,4,4 )
pole = Pole( 1,2,3,4)
field.setVertex( 1,1,1, pole )
poleOut = field.getVertex(1,1,1)
np.testing.assert_almost_equal( poleOut.getValue(),  4,       err_msg="Failed case 3", verbose=False )
np.testing.assert_almost_equal( poleOut.getVector(), [1,2,3], err_msg="Failed case 4", verbose=False )

print( "ScalarField() tests succeeded" )

## Implement createScalarField() 

createScalarField() should generate a scalar field with the ScalarField class, use the superQuadric function to compute the scalar values from the positions of the grid points, then store results as a pole on each grid point of the field.  Since we're only handling a single superquadric primitive centered at the origin, and rotations are not considered, we'll provide arguments to the superquadric function too.

Implement the createScalarField() function in the cell below.  It should accept the following arguments:

- __cellSize__: (scalar) Edge length of each cell in the scalar field (i.e. distance between grid points). default = 0.25.
- __bbmin__: (3D vector) position coordinate defining the most negative corner of the field (default = [-5,-5,-5]
- __bbmax__: (3D vector) position coordinate defining the most positive corner of the field (default = [5,5,5]).
- __scale__: (3D vector) Local axis scale factors for the superquadric primitive (default = [1,1,1])
- __exponents__: (3D vector) Exponents for the superquadric function (default = [2,2,2]).

The function should return the object representing the fully created scalar field, or __None__ upon failure.

In [ ]:
#===================================================================================
# createScalarField() - Creates a scalar field using superquadric() to assign 
#                       scalar values to the points as poles
#
# Parameters:
# cellsize: (scalar) dimension of a cell in the scalar field.
# bbmin: (3D vector) location of the lower corner of the scalar field
# bbmax: (3D vector) location of the upper corner of the scalar field.
# scale: (3D vector) scaling factor for the superquadric primitive
# exponents: (3D vector) exponents for the superquadric primitive
#
# Returns:
# scalar field: (object) the resulting scalar field object, or None upon error.
#===================================================================================
def createScalarField( cellSize=0.25, bbmin=[-5,-5,-5], bbmax=[5,5,5], scale=[1,1,1], exponents=[2,2,2] ):

    # your code here
    
    return( None )

In [ ]:
# testing.  Set flag for test mode

test_mode = 0       # 0 = low resolution scalar field, 1 = high resolution scalar field

In [ ]:
### test cases ###

if  test_mode == 0:
    field = createScalarField()

    pole = field.getVertex( 20, 20, 20 )
    # print( "v: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [0.0,0.0,0.0], decimal=6, err_msg="Failed case 2", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 0.0, decimal=6, err_msg="Failed case 2", verbose=False )
    print( "Test 1 - center grid point: Passed" )

    pole = field.getVertex( 40, 40, 40 )
    # print( "v: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [5,5,5], decimal=6, err_msg="Failed case 2", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 8.660254037844387, decimal=6, err_msg="Failed case 2", verbose=False )
    print( "Test 2 - corner grid point: Passed" )

    field = createScalarField( 0.667, [-5,-5,-5], [5,5,5], [4,2,1], [0.1, 8, 4] )
    pole = field.getVertex( 3, 2, 2 )
    # print( "v: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [-2.857142857142857, -3.5714285714285716, -3.5714285714285716], decimal=6, err_msg="Failed case 3", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 16.34177612693006, decimal=6, err_msg="Failed case 3", verbose=False )
    print( "Test 3 - random position: Passed" )

    pole = field.getVertex( 13, 9, 4 )
    # print( "v: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [4.2857142857142865, 1.4285714285714288, -2.1428571428571432], decimal=6, err_msg="Failed case 3", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 4.7074035414110575, decimal=6, err_msg="Failed case 3", verbose=False )
    print( "Test 4 - random position: Passed" )

elif test_mode == 1:
    # #### hi-def field ###
    print( "-------- hi def field ---------" )
    # # [300,150,245]
    field = createScalarField( 0.05, [-5, -2, 0], [10, 5.5, 12.25], [6,2,7.5], [0.1, 80, 4] )

    pole = field.getVertex( 150, 75, 123 )
    print( "v0: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [2.5, 1.75, 6.15], decimal=6, err_msg="Failed case 2", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 1.1697523792416138, decimal=6, err_msg="Failed case 2", verbose=False )
    print( "Test 1 - center: Passed" )

    pole = field.getVertex( 0, 0, 0 )
    print( "v1: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [-5.0, -2.0, 0.0], decimal=6, err_msg="Failed case 2", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(),  1.4078114378573263, decimal=6, err_msg="Failed case 2", verbose=False )
    print( "Test 2a - lower corner: Passed" )

    pole = field.getVertex( 300, 150, 245 )
    print( "v2: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [10.0, 5.5, 12.25], decimal=6, err_msg="Failed case 2", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 374375787445778100, decimal=2, err_msg="Failed case 2", verbose=False )
    print( "Test 2b - upper corner: Passed" )

    pole = field.getVertex( 137, 21, 85 )
    print( "v3: ", pole.getVector(), pole.getValue() )
    np.testing.assert_array_almost_equal( pole.getVector(), [1.8500000000000005, -0.95, 4.25], decimal=6, err_msg="Failed case 3", verbose=False )
    np.testing.assert_almost_equal( pole.getValue(), 0.9960486598671837, decimal=6, err_msg="Failed case 3", verbose=False )
    print( "Test 3 - random position: Passed" )

print( "scalar field tests succeeded" )

# Part 4: Marching Tetrahedra 6

Marching Tetrahedra is an extension of Marching Cubes where each cell of the scalar field is subdivided into tetrahedra.  Tetrahedra are simpler to evaluate for intersections with the isosurface and avoid ambiguities that arise in Marching Cubes.  The algorithm comes in two versions: 5 tetrahedra and 6 tetrahedra.  In this section we'll implement Marching Tetrahedra 6.

One advantage to Marching Tetrahedra is there are far fewer unique arrangements of positive poles to consider as tetrahedra have fewer edges.  Marching Cubes has 256 overall cases, but only 22 are unique.  Marching Tetrahedra has 16 cases, but only 5 are unique (only 3 producing geometry), and no ambiguities to resolves because no matter how a edges are bisected, the result is always a triangle with clear orientation.

One disadvantage of the six tetrahedra method is it takes longer to compute, and generates more triangles, edges, and vertices in the isosurface.  The additional geometry does not necessarily improve the detail of the isosurface.

## Table driven programming

As discussed in Wenger's book, computing intersections for tetrahedra is fairly straightforward.  However, since there are a limited number of configurations of positive vertices, it means we know all the possible outcomes in advance.  Therefore, we can gain a performance advantage by precomputing the results and inserting them into a table, then looking up the correct solution as needed at runtime.

Below are the lookup tables for Marching Tetrahedra 6 to assist the construction of the isosurface when traversing the cells.  Later in the project you will be responsible for constructing the equivalent tables for Marching Tetrahedra 5 and Marching Cubes.  The tables can certainly be changed to other data structures or formats, but the important part is the relationships of the values in the table.  So spend some time getting familiar with them.  It may not make sense until later in the project when implementing the classes.

The lookup table is divided into three primary data structures: 
- Cell edge vertex indices.
- Tetrahedra vertex indices.
- Polygon connectivity.

__aIndicesSource6__ and __aIndicesTarget6__ describe the vertex pairs that form an edge in the cell.  It is used as a convenience to march around the cell testing for intersections with the isosurface, and for copying vertex indices between cells.  Given an edge index 'n', aIndicesSource6[n] represents the first cell vertex index of the edge, and aIndicesTarget6[n] represents the second cell vertex index of the edge.

__aTetraIndices6__ defines the tetrahedra of the cell.  Each index of this table contains the indices of the 4 cell vertices which define a tetrahedron.  The indices are always listed in ascending order.  This means they do not describe the direction polygons wind on the tetrahedra (nor is that important).

__aTriangleIndices6__ defines the order in which edges should be connected to form triangles based on the bitcode for the tetrahedra (explained later).  The row specifies the tetrahedron in the cell.  The column specifies which of the tetrahedron configurations is used to create the polygon.  The first 4 columns are 1-pole cases, the last 3 columns are 2-pole cases.

In [ ]:
### lookup tables for Marching Tetrahedra ###

# Each version of Marching tetrahedra has 4 lookup tables:
#   tetra indices:  Indices of the vertices forming each tetrahedra in the cell
#   source indices: Indices of the first vertex of each edge in the cell
#   target indices: Indices of the second vertex of each edge in the cell
# triangle indices: Indices of the *edges* containing vertices (intersections) 
#                   which form the triangle(s) of the isosurface.
#
# NOTE: the vertex and edge indices are local to the cell for consistency.
#       It is the developer's responsibilty to correlate the indices to the isosurface.
#
# Tetra indices are used each time the cell is called to evaluate
# and limit the scope of evaluation to the current tetrahedron.
#
# Each call to evalCell() generates 5 or 6 calls to evalTetra() 
# depending on which Marching Tetrahedra algorithm is in use.
# This affects the arrangement of tetrahedra within the cell and requires
# each case be handled differently.  Furthermore, in the case of 5 tetrahedra,
# the orientation of the edges within the cell changes every other cell.
# Two sets of indices must be devised for the 5 tetrahedra case.
# The two cases are specified as 5A and 5B (rotated cell).
#
# Triangle edge indices are arranged as a table where the row indicates the tetrahedron,
# and the column indicates the configuration which needs to be resolved.
# the first four are single pole cases, and the last three are two-pole cases.
# There are only three two-pole situations because some are reflections of others (i.e. same logic).
# The returned list contains the indices of the edges within the cell that contain
# intersections which must be connected to form the triangles representing the isosurface for that tetrahedron.
# The edge indices are correlated to the source and target indices tables repsectively.
#
# Example:
#   triangle edge index 14 represents the edge formed by connecting source vertex 14 and target vertex 14.
#   For Marching tetrahedra 5A = edge formed by vertices [1,6]
#   For Marching tetrahedra 5B = edge formed by vertices [2,5]
#   For Marching tetrahedra 6  = edge formed by vertices [0,7]

#--------------------------
# 6 tetrahedra
#--------------------------
#                                        1 1  1 1 1  1 1 1
#                  0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource6 = [0,1,2,3, 4,5,6,7, 0,1,2,3, 0,0,0, 6,6,6, 0] # edge start vertex
aIndicesTarget6 = [1,2,3,0, 5,6,7,4, 4,5,6,7, 2,5,7, 4,1,3, 6] # edge end vertex

aTetraIndices6 = [
    [0,4,6,7], # tetra 0
    [0,3,6,7], # tetra 1
    [0,2,3,6], # tetra 2
    [0,1,2,6], # tetra 3
    [0,1,5,6], # tetra 4
    [0,4,5,6]  # tetra 5
]

# triangle indices winding order 6 tetrahedra
aTriangleIndices6 = [
    #  1-pole and 3-poles                          2-poles
    #[ [v0],    [v1],      [v2],      [v3]         [0011, 1100]   [0101, 1010]  [0110, 1001] ]
    [ [14,18,8], [8,15,7],  [6,15,18], [14,7,6],   [18,15,7,14],  [14,6,15,8],  [18,6,7,8]   ], # tetra 0
    [ [3,18,14], [11,17,3], [18,17,6], [6,11,14],  [18,14,11,17], [3,17,6,14],  [11,6,18,3]  ], # tetra 1
    [ [18,3,12], [12,2,10], [17,2,3],  [18,10,17], [3,2,10,18],   [18,17,2,12], [3,17,10,12] ], # tetra 2
    [ [18,12,0], [0,1,16],  [10,1,12], [18,16,10], [18,12,1,16],  [18,10,1,0],  [12,10,16,0] ], # tetra 3
    [ [0,13,18], [16,9,0],  [13,9,5],  [5,16,18],  [18,16,9,13],  [18,0,9,5],   [0,16,5,13]  ], # tetra 4
    [ [18,13,8], [8,4,15],  [5,4,13],  [18,15,5],  [13,4,15,18],  [18,5,4,8],   [13,5,15,8]  ]  # tetra 5
]

## Implement the Cell class

The Marching Cubes and Marching Tetrahedra algorithms get their name by the fact the algorithm marches through a scalar field and evaluates each cell to produce an isosurface.  This implies we must have a cell to evaluate, which we haven't created yet.  In this section we'll implement the __Cell__ class to represent a cell in the scalar field.

A cell is a virtual cube surrounding an arbitrary location in the scalar field that isn't a grid point.  It has 8 vertices, and in the case of Marching Tetrahedra 6, has 19 edges.  The first 12 edges define the shape of the cube, and the last 7 edges represent the diagonals on the faces defining the arrangement of tetrahedra.  But you say, "wait a minute - a cube only has 6 diagonals!".  To that you would be correct.  The 7th diagonal runs from the lower-left-front vertex to the upper-right-rear vertex (opposite corners).  All the tetrahedra in the cell share this diagonal as a common edge, which makes Marching Tetrahedra 6 an easier algorithm to implement, and why we are doing it first.

If we evaluate each cell in isolation, the resulting isosurface will be a collection of triangles casually known as a 'polygon soup'.  That is, the generated triangles will collectively form a quilt of the isosurface, but they will not share any vertices or edges to form a contiguous surface.  Polygon soups are inefficient because they require more storage space due to duplication of features.  They also present problems when manipulated as the surface tends to split easily because the triangles are not connected, and is especially poor during rendering as numerical imprecision make the polygon mesh appear to have slivers/cracks between adjacent triangles.

The Cell class will be responsible for keeping track of which edges contain vertices (intersections) inserted by the Marching algorithm so the triangles from adjacent cells can be stitched together to form a continguous polygon mesh where all the edges and vertices are shared.  To do this, the components of the cell must be numbered consistently to make the algorithm work.  We'll use the following vertex and edge ordering:

![Marching Tetrahedra 6](./images/tetra_6.png)

The edges are numbered in yellow, and the vertices numbered in green.  Of special note is edge 18 which spans opposite corners of the cell.  This edge is shared by all tetrahedra in the cell.

When the eye point is placed at the lower corner and looking at the upper corner of the main diagonal, you'll see the silhouettes of the 6 tetrahedra arranged around the main diagonal (edge 18), and indexed in counter-clockwise order (see lookup table).  Use this intuition to guide your understanding of the cell's (and tetrahedra's) role within the algorithm.  Consult the lookup tables to see how the tetrahedra are defined relative to this view.

![Marching Tetrahedra 6](./images/tetra_6_view.png)

The Cell class should include the following properties:
- __Count__: Active count of number of vertices in the cell with pole values
- __values__: list/array of pole values.
- __positions__: list/array of pole positions.
- __edgeMap__: list where each index represents an edge of the cell. An index's value is the vertex index inserted by the Marching algorithm, or -1 if no vertex stored on that edge.

The Cell class should include the following methods:

- __\_\_init\_\_()__: initializes the cell's properties
- __addPole( pole )__: Inserts a pole at the next available corner of the cell.
- __copyEdgeMap( cellU, cellV, cellW )__: copies vertex indices from adjacent cell(s) edges to the current cell's edges where the edges are common to both cells.

For the copyEdgeMap() method, the arguments cellU, cellV, and cellW are the adjacent cells bordering the left, front, and bottom sides respectively of the current cell.  Depending on the location within the scalar field, these adjacent cells may not be available.  copyEdgeMap() must recognize these situations and react accordingly.  A better intuition may be developed how to implement this feature when implementing the IsoMesh class in the next section.

In [ ]:
#====================================================
# Cell()
#
# cube of a scalar field recording the vertex values
# and list of which edges contain inserted vertices.
#====================================================
class Cell( object ):
    def __init__( self, count=0, values=[], positions=[] ):
        self.Count     = 0
        self.values    = []
        self.positions = []
        self.edgeMap   = [-1,-2,-3,-4, -5,-6,-7,-8, -9,-10,-11,-12, -13,-14,-15, -16,-17,-18, -19]

    # === Methods ===
    #-------------------------------------------------------------------------------------
    # clear() - resets the cell to default values
    #-------------------------------------------------------------------------------------
    def clear( self ):
        return( None )
        
    #-------------------------------------------------------------------------------------
    # addPole() - Records pole object 'p' at the next available vertex of the cell.
    #-------------------------------------------------------------------------------------
    def addPole( self, p ):
        return( None )

    #-------------------------------------------------------------------------------------
    # copyEdgeMap() - copies poles of vertices in common from specified adjacent cells to the current cell.
    #-------------------------------------------------------------------------------------
    def copyEdgeMap( self, oCelli, oCellj, oCellk ):
        return( None )    


In [ ]:
## test cases ###

# pole test
oCell = Cell()
n = 10
values    = []
positions = []
for i in range( 0, n ):
    v = np.random.rand()
    p = Pole( i+1, i+2, i+3, v )
    values.insert( i, v )
    positions.insert( i, [i+1,i+2,i+3] )
    oCell.addPole( p )
np.testing.assert_almost_equal( oCell.Count, n, err_msg="Failed case 1", verbose=False )
np.testing.assert_almost_equal( oCell.values, values, err_msg="Failed case 2", verbose=False )
np.testing.assert_array_almost_equal( oCell.positions, positions, err_msg="Failed case 3", verbose=False )
    
# edgemap test
cellA = Cell()
cellB = Cell()
cellC = Cell()

for i in range( 0, 19 ):
    cellA.edgeMap[i] = i
    cellB.edgeMap[i] = i
    cellC.edgeMap[i] = i

oCell = Cell()
oCell.edgeMap = [-1,-2,-3,-4, -5,-6,-7,-8, -9,-10,-11,-12, -13,-14,-15, -16,-17,-18, -19]
oCell.copyEdgeMap( cellA, cellB, cellC )
np.testing.assert_almost_equal( oCell.edgeMap, emValues, err_msg="Failed case 4", verbose=False )
print( "cell tests succeeded" )

## Implement IsoMesh class

This class implements the Marching Tetrahedra algorithm and contains methods to generate the isosurface.  The general workflow is:
- Initialize the isomesh with a scalar field.
- Call isomesh.createIsoSurface() to traverse the scalar field cell-by-cell and copying relevant data into a cell object which is passed to isomesh.evalCell().  The cell is recorded in the isomesh.Cells list.
- Isomesh.evalCell() computes an 8-bit bitcode by examining the vertices of the cell.  The bitcode is used to determine where the isosurface intersects the cell's edges as the vertices sharing such an edge will have different bit values.
- Isomesh.evalCell() inserts the vertices into the isomesh.Vertices list and records the insertion in the cell.  The cell is passed onto isomesh.evalTetra() for each tetrahedron in the cell.
- Isomesh.evalTetra() remaps the 8-bit bitcode of the cell to a 4-bit bitcode of the current tetrahedron.  The bitcode determines where triangles should be drawn, inserting triangles into the isomesh.Polygons list were applicable while referencing appropriate vertices from the cell.

Some additional details are provided below.  The rest are left as an exercise.

The IsoMesh class should define the following properties:
- __VertexCount__: (integer) Number of vertices in the isosurface mesh.
- __Vertices__: (list) vertex positions of the isosurface (the vertex list).  Each index is a 1D array of 3 scalars (i.e. a 3D vector)
- __PolygonCount__: (integer) Number of triangles in the isosurface mesh.
- __Polygons__: (list) Each index is a 1D array of integers representing a polygon in the isosurface.  Each polygon is described as the number of sides in the polygon, followed by the indices of the vertices comprising the polygon.  The first and last vertex in the list are automatically connected to form a closed polygon.
- __CellCount__: (integer) Marker for the current index in the Cells list.
- __CellsU__: (integer) number of cells in the U (X) direction of the scalar field ( left   -> right ).
- __CellsV__: (integer) number of cells in the V (Y) direction of the scalar field ( front  -> back  ).
- __CellsW__: (integer) number of cells in the W (Z) direction of the scalar field ( bottom -> top   ). 
- __Cells__: (list) 1D representation of a flattened 3D array of cells evaluated so far.

The IsoMesh class should define the following methods:
- __clear()__: Resets the object to default settings.
- __addCell( cell )__: appends 'cell' to the Cells list, and returns it's index to the caller.
- __getCell( index )__: returns the cell at the specified index from the Cells list.
- __addVertex( [x,y,z] )__: appends the specified position to the vertex list, and returns it's index to the caller.
- __getVertexPosition( index )__: returns the vertex position at the specified index of the vertex list.
- __insertVertex( isovalue, p1, p2, v1, v2 )__: given an isovalue and two vertices of the cell along a common edge, compute where the isosurface intersects the edge.  Add the resulting vertex position to the vertex list, and return it's index in the vertex list to the caller.  p1, p2 are the scalar values of the edge vertices, and v1, v2 are the vertex positions.
- __addPolygon( [N, v0, v1, .., vN-1] )__: append polygon description to the polygon list, return it's index in the Polygons list to the caller.
- __createIsoSurface( Field, isovalue )__: Traverse the scalar field and evaluate at the specified isovalue. Returns the isosurface as a polygon mesh, or __None__ if no isosurface produced.
- __evalCell( cell, isovalue )__: Computes 8-bit bitcode for the cell specifying which vertices are positive, then determines which edges intersect with the isosurface inserting vertices at the intersections. Calls evalTetra() to define the triangles where the isosurface intersects the tetrahedron.
- __evalTetra( tetraIndex, tetraIndices, bitCode, edgeMap )__: Converts the 8-bit bitcode of the cell to a 4-bit bitcode of the tetrahedron to determine the positive vertices, then builds polygon description(s) for the triangles generated by connecting the inserted vertices, and adds the polygons to the IsoMesh's polygon list.
  - __tetraIndex__: index of the tetrahedron within the cell.  
  - __tetraIndices__: cell's vertex indices comprising the tetrahedron.  
  - __bitcode__: 8-bit code from evalCell().
  - __edgeMap__: cell's edgemap which records the vertices inserted into the cell's edges.
- __createMesh()__: Creates a polygon mesh from the Vertices and Polygons lists, then returns it to the caller as a Pyvista.PolyData object, or __None__ on failure.

__NOTE__: Additional properties/methods may be added if desired, but are not necessary.

In [ ]:
#=============================================================================
# IsoMesh() - class representing the isosurface generated by Marching Tetrahedra.
#
#=============================================================================
class IsoMesh( object ):
    def __init__( self, cellsU=0, cellsV=0, cellsW=0 ):
        self.VertexCount   = 0
        self.Vertices      = []        
        self.PolygonCount  = 0
        self.Polygons      = []
        self.CellCount     = 0
        self.CellsU        = cellsU     # input scalar field dimensions
        self.CellsV        = cellsV
        self.CellsW        = cellsW
        self.CellSheetSize = ( cellsU * cellsV )
        self.Cells         = [None] * ( self.CellSheetSize + 1 )     

        self.Pow           = [1,2,4,8,16,32,64,128]        # precomputed powers of 2 for bitcode generation

    # === Methods ===
    #=======================================================
    # clear()
    #=======================================================
    def clear( self ):
        self.VertexCount   = 0
        self.PolygonCount  = 0
        self.CellCount     = 0
        self.Vertices      = []
        self.Polygons      = []
        self.Cells         = []
        self.CellsU        = 0
        self.CellsV        = 0
        self.CellsW        = 0
        
    #=======================================================
    # addCell() - append cell into cell list
    #=======================================================
    def addCell( self, oCell ):
        return( None )

    #=======================================================
    # getCell() - retrieve the cell at specified index
    #=======================================================
    def getCell( self, i=0, j=0, k=0 ):
        return( None )

    #=======================================================
    # addVertex() - add vertex (x,y,z) to the vertex list
    #=======================================================
    def addVertex( self, x=0, y=0, z=0 ):
        return( None )
    
    #=======================================================
    # insertVertex() - generate a vertex at location where isovalue crosses edge p1-p2
    # then insert into the mesh.
    #=======================================================
    def insertVertex( self, isovalue, p1, p2, v1, v2 ):
        return( None )       
    
    #=======================================================
    # addTriangle() - add triangle to the triangle list
    #=======================================================
    def addTriangle( self, v1, v2, v3 ):
        return( None )

    #=======================================================
    # addPolygon() - add polygon to the polygon list
    #=======================================================
    def addPolygon( self, edges ):
        return( None )

    #=======================================================
    # createMesh() - given a list of vertices and list of polygons, generate a polygon mesh.
    #=======================================================
    def createMesh( self ):
        
        print( "nb[vertices,polygons]: ", self.VertexCount, self.PolygonCount )      
        print( "createMesh[vertices,polygons]: ", len(self.Vertices), len(self.Polygons) )
        
        if ( len(self.Vertices) < 3 or len(self.Polygons) < 4 ):
            # no mesh data
            print( "ERROR - empty mesh" )
            return( None )

        sw = StopWatch(0)

        try:
            mesh = pv.PolyData( np.array( self.Vertices ), np.hstack( self.Polygons ) )
        except TypeError:
            print( "ERROR: Cannot create mesh: ", TypeError )
            return( None )

        print( "   createMesh(): ", sw.GetTotal(), " seconds" )
        ### for debugging only (lot of output) ###
#         for i in range(0, len( self.Vertices) ):
#             print( "v[" + str(i) + "]: " + str( self.Vertices[i] ) )
#         for i in range(0, len( self.Polygons) ):
#             print( "p[" + str(i) + "]: " + str( self.Polygons[i] ) )
            
        return( mesh  )

    #=======================================================
    # createIsoSurface() - generate an isosurface at the specified isovalue
    #=======================================================
    def createIsoSurface( self, oField, isovalue ):
        return( None )

    #=======================================================
    # evalCell() - compute location and orientation of polygons for the cell.
    #=======================================================
    def evalCell( self, oCell, isovalue, coord ):
        return( None )

    #=======================================================
    # evalTetra() - evaluate tetrahedron
    #=======================================================
    def evalTetra( self, tetraIndex, aIndices, bitCode, edgeMap ):
        return( None )
    

In [ ]:
# display results
cellSize = 0.25
isovalue = 1.0

field       = createScalarField( cellSize, [-5,-5,-5], [5,5,5], [1,1,1], [4,4,4] )
isoMesh     = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, isovalue )

polygonMesh.plot( scalars=np.arange( len( polygonMesh.points ) ), show_edges=True, cpos=[0.5,0.25,0.25] )

In [ ]:
## test cases ##
field = createScalarField( 1.0, [-5,-5,-5], [5,5,5], [1,1,1], [4,4,4] )
isoMesh = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, 1.0 )

np.testing.assert_equal( len(polygonMesh.points), 14, err_msg="Incorrect vertex count",  verbose=False )
np.testing.assert_equal( polygonMesh.n_faces,     24, err_msg="Incorrect polygon count", verbose=False )

##
field = createScalarField( 0.25, [-5,-5,-5], [5,5,5], [4,4,4], [4,4,4] )
isoMesh = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, 1.0 )

np.testing.assert_equal( len(polygonMesh.points), 19022, err_msg="Incorrect vertex count",  verbose=False )
np.testing.assert_equal( polygonMesh.n_faces,     38040, err_msg="Incorrect polygon count", verbose=False )
print( "IsoMesh tests succeeded" )

# Part 5: Marching Tetrahedra 5

Well, that was easy wasn't it? (Just kidding).  In this section we'll tweak the classes created above to support Marching Tetrahedra 5.

Marching Tetrahedra 5 is a variant of the Marching Tetrahedra algorithm which subdivides the cell into 5 tetrahedra instead of 6.  This algorithm is more efficient as fewer triangles are generated, and less computation occurs.  However, to get those benefits requires a few trade offs which make the algorithm a bit more complicated.

If you subdivde the cell into 5 tetrahedra by drawing diagonals along each edge in an alternating pattern as you walk around the cell, you'll notice two tetrahedra can be formed at the bottom of the cell at opposite corners, and two more tetrahedra can be formed at the top of the cell also at opposite corners, but perpendicular to the bottom two tetrahedra.  This conveniently leaves a void in the middle of the cell which happens to be the 5th tetrahedron.

The challenge is the diagonals of the cell do not align with the diagonals of the adjacent cells.  Fortunately, inspection reveals rotating the cell by 90 degrees on the vertical axis will resolve the problem.  This means we must rotate every other cell in the scalar field by 90 degrees to form a cohesive isosurface, which can be a bit of a headache to manage.  The cell's vertices and edges will be numbered the same as in Marching Tetrahedra 6, but the 19th edge (e18) no longer exists, and some of the diagonals will have an alternate orientation forming a zig-zag as you walk around the cell (numbering is not affected by change in orientation).

## Lookup tables for Marching Tetrahedra 5. 

Since there is a 90 degree rotation every other cell in the field, we must develop two sets of lookup tables to account for each of the two cell orientations.  We'll discern between the two orientations of the cell with suffixes.  We'll use __5A__ to indicate the cell in default orientation, and __5B__ to indicate the cell has been rotated by 90 degrees:

![Marching Tetrahedra 5](./images/tetra_5.png)

In both cases, the numbering of the vertices and edges will be according to world coordinates, not local cell coordinates.  This means vertex 0 is the lower-left-front vertex regardless of how the cell is oriented.  For the most part, the indexing for the individual cell vertices does not change, but the pairings defining the edges and tetrahedra do.  You will need to think this through.

Append the tables for Marching Tetrahedra 5 to the lookup tables in the cell below.  The simplest approach would be to duplicate the tables replacing the '6' suffix with '5A' and '5B', then modify entries as needed.

In the IndicesSource and IndicesTarget tables, each index stores the vertex index comprising the cell's edge endpoints.
In the TetraIndices table, define the cell vertices which comprise each of the 5 tetrahedron (in ascending order).
In the TriangleIndices table, define the triangles so the normals point towards the selected vertices.

Generating the tables is not difficult, but debugging can get quite tedious.  __TIP__: draw a diagram.

In [ ]:
### lookup tables for Marching Tetrahedra ###

# Each version of Marching tetrahedra has 4 lookup tables:
#   tetra indices:  Indices of the vertices forming each tetrahedra in the cell
#   source indices: Indices of the first vertex of each edge in the cell
#   target indices: Indices of the second vertex of each edge in the cell
# triangle indices: Indices of the *edges* containing vertices (intersections) 
#                   which form the triangle(s) of the isosurface.
#
# NOTE: the vertex and edge indices are local to the cell for consistency.
#       It is the developer's responsibilty to correlate the indices to the isosurface.
#
# Tetra indices are used each time the cell is called to evaluate
# and limit the scope of evaluation to the current tetrahedron.
#
# Each call to evalCell() generates 5 or 6 calls to evalTetra() 
# depending on which Marching Tetrahedra algorithm is in use.
# This affects the arrangement of tetrahedra within the cell and requires
# each case be handled differently.  Furthermore, in the case of 5 tetrahedra,
# the orientation of the edges within the cell changes every other cell.
# Two sets of indices must be devised for the 5 tetrahedra case.
# The two cases are specified as 5A and 5B (rotated cell).
#
# Triangle edge indices are arranged as a table where the row indicates the tetrahedron,
# and the column indicates the use case which needs to be resolved.
# the first four are single pole cases, and the last three are two-pole cases.
# There are only three two-pole situations because some are reflections of others (i.e. same logic).
# The returned list contains the indices of the edges within the cell that contain
# intersections which must be connected to form the triangles representing the isosurface for that tetrahedron.
# The edge indices are correlated to the source and target indices tables repsectively.
#
# Example:
#   triangle edge index 14 represents the edge formed by connecting source vertex 14 and target vertex 14.
#   For Marching tetrahedra 5A = edge formed by vertices [1,6]
#   For Marching tetrahedra 5B = edge formed by vertices [2,5]
#   For Marching tetrahedra 6  = edge formed by vertices [0,7]


#--------------------------
# 5 tetrahedra
#--------------------------
#                                         1 1  1 1 1  1 1 1
#                   0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource5A = []
aIndicesTarget5A = []

# tetrahedra vertex indices winding order 5A
aTetraIndices5A = [
]

# triangle edge indices winding order 5A
aTriangleIndices5A = [
    #  1-pole and 3-poles                             # 2-poles
    #[  [v0],     [v1],       [v2],       [v3]        [0011, 1100]   [0101, 1010]  [0110, 1001] ]

]

#                                         1 1  1 1 1  1 1 1
#                   0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource5B = []
aIndicesTarget5B = []

# tetrahedra vertex indices winding order 5B
aTetraIndices5B = [
]

# triangle edge indices winding order 5B
aTriangleIndices5B = [
    #   1-pole and 3-poles                            # 2-poles
    #[  [v0],     [v1],       [v2],       [v3]        [0011, 1100]   [0101, 1010]  [0110, 1001]    ]

]


#--------------------------
# 6 tetrahedra
#--------------------------
#                                        1 1  1 1 1  1 1 1
#                  0 1 2 3  4 5 6 7  8 9 0 1  2 3 4  5 6 7
aIndicesSource6 = [0,1,2,3, 4,5,6,7, 0,1,2,3, 0,0,0, 6,6,6, 0] # edge start vertex
aIndicesTarget6 = [1,2,3,0, 5,6,7,4, 4,5,6,7, 2,5,7, 4,1,3, 6] # edge end vertex

aTetraIndices6 = [
    [0,4,6,7], # tetra 0
    [0,3,6,7], # tetra 1
    [0,2,3,6], # tetra 2
    [0,1,2,6], # tetra 3
    [0,1,5,6], # tetra 4
    [0,4,5,6]  # tetra 5
]

# triangle edge indices winding order 6 tetrahedra
aTriangleIndices6 = [
    #  1-pole and 3-poles                          # 2-poles
    #[ [v0],    [v1],      [v2],      [v3]         [0011, 1100]   [0101, 1010]  [0110, 1001] ]
    [ [14,18,8], [8,15,7],  [6,15,18], [14,7,6],   [18,15,7,14],  [14,6,15,8],  [18,6,7,8]   ], # tetra 0
    [ [3,18,14], [11,17,3], [18,17,6], [6,11,14],  [18,14,11,17], [3,17,6,14],  [11,6,18,3]  ], # tetra 1
    [ [18,3,12], [12,2,10], [17,2,3],  [18,10,17], [3,2,10,18],   [18,17,2,12], [3,17,10,12] ], # tetra 2
    [ [18,12,0], [0,1,16],  [10,1,12], [18,16,10], [18,12,1,16],  [18,10,1,0],  [12,10,16,0] ], # tetra 3
    [ [0,13,18], [16,9,0],  [13,9,5],  [5,16,18],  [18,16,9,13],  [18,0,9,5],   [0,16,5,13]  ], # tetra 4
    [ [18,13,8], [8,4,15],  [5,4,13],  [18,15,5],  [13,4,15,18],  [18,5,4,8],   [13,5,15,8]  ]  # tetra 5
]

## Modify the Cell class

The __Cell__ class must be updated to recognize Marching Tetrahedra 5.  Fortunately, the change is simple.  Only the copyEdgeMap() method requires modification:

__copyEdgeMap( cellU, cellV, cellW, nbTetra )__

An argument `nbTetra` has been added to indicate how many tetrahedra the cell will be subdivided.  It can accept a value of 5 or 6.  If 5 is specified, copy the edges using the Marchng TetraHedra 5 rules.  If 6 is specified, copy the edges using the Marching Tetrahedra 6 rules devised earlier.  If any other value is specified, return without doing any work.

If the lookup tables were defined correctly, mapping of edges between adjacent cells should be the same for both cell orientations of Marching Tetrahedra 5.

In [ ]:
### copy and paste your 'Cell' class here, then make the necessary modifications.

## Modify the IsoMesh class

Most of the class remains unchanged, but a few of the more important methods require some modifications.

- __createIsoSurface()__: Must account for cell orientation while traversing the scalar field using the Marching Tetrahedra 5 algorithm.  This affects how cells are defined, and edges copied between adjacent cells.
- __evalCell()__: Must account for tetrahedra count and cell orientation to read the correct lookup tables before calling evalTetra().
- __evalTetra()__: Must account for tetrahedra count and cell orientation to read the correct lookup tables.

__NOTE__: Other changes may also be necessary.

In [ ]:
### copy and paste your 'IsoMesh' class here, then make the necessary modifications.

In [ ]:
# display results
bbmin = [-5,-5,-5]
bbmax = [5,5,5]
scale = [4,4,4]
exponents = [4,4,4]

cellSize = 0.5
isovalue = 1.0
method = 5

field       = createScalarField( cellSize, bbmin, bbmax, scale, exponents )
isoMesh     = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, isovalue, method )

polygonMesh.plot( scalars=np.arange( len( polygonMesh.points ) ), show_edges=True, cpos=[0.1,0.25,0.1] )

print( "nb[vertices,polygons]: ", len(polygonMesh.points), polygonMesh.n_faces )

In [ ]:
## test cases
field = createScalarField( 0.25, [-5,-5,-5], [5,5,5], [1,1,1], [2,2,2] )
isoMesh = IsoMesh()
polygonMesh = isoMesh.createIsoSurface( field, 1.0 )

#
np.testing.assert_almost_equal( len( polygonMesh.points ), 1800, err_msg="Incorrect vertex count",  verbose=False )
np.testing.assert_almost_equal( polygonMesh.n_faces, 1256, err_msg="Incorrect polygon count", verbose=False )
print( "tests succeeded" )

# Part 6: Marching Cubes

Now that you have a grasp of the overall algorithm as defined by the variants, it's time to implement the original.

Marching Cubes doesn't require too many more modifications, and in many ways is simpler than Marching Tetrahedra.  IsoMesh.evalTetra() is no longer required, and Cell.copyEdgeMap() only needs to account for changes in how edges are mapped between adjacent cells in the same manner as resolved earlier.

However, not all comes up roses as the lookup table is considerably larger, and ambiguity arises in resolving many of the cases.  This can be handled with computation, but it's usually more efficient and reliable to resolve the ambiguities in the lookup table.  IsoMesh.evalCell() will need to be modified to account for the fact many cases will generate multiple triangles, as well as to maneuver around the code already in place for Marching Tetrahedra when it is not required.


![Marching Cubes cell configuration](./images/cube_config.png)

## Lookup table

There are now 256 cube configurations in the lookup table vs. 7 for Marching Tetrahedra, and many cases consist of multiple triangles.

Define a table called __aCaseMap__ with 256 entries.  The first and last entry are not used as they represent the configurations of all positive vertices or no positive vertices resulting in no isosurface, but you must still provide a valid polygon description to prevent errors from being thrown in case something falls through the cracks.  All other cases draw at least one triangle.

Each entry in the table represents a configuration of positive vertices in the cell according to the bit code.  For example, if only vertex 3 is a positive vertex, the bit code would be 00001000 as expressed in binary, which evaluates to 8 in decimal.  The polygon description of the triangles drawn for that configuration should then be recorded at index 8 in the table.  The polygon description should be as defined in part 1, but with one important alteration - the vertex indices will be replaced with the cell's edge indices which contain the vertices to be connected (a double indirection).  However, in cases where multiple triangles are to be drawn, they must be appended in succession:
```
aCaseMap[
  [3,1,2,3],                  # 1 triangle with vertices [1,2,3].
  [3,1,2,3,3,4,5,6,3,7,8,9],  # 3 triangles appended one after the other with 
                                     vertices [1,2,3], [4,5,6], and [7,8,9].
    
                            # NOTE: must replace vertex indices with edge indices to support indirection.
                            #   Example: vertices 0 and 1 are positive.  bitcode: 00000011, case index 3.
  [3,9,8,3,3,3,1,9],        #   <-- 2 triangles defined by vertices inserted on edges [9,8,3] and [3,1,9],
                            #   with normals facing vertices 0 and 1 while respecting the right-hand rule.
                                
]
```

![Marching Cubes polygon description](./images/cube_connections.png)

Since the cell contains no diagonals in Marching Cubes, the edge indices will range from 0 to 11 and be ordered in the same manner as used previously.  The lists must be parsed to lookup the vertex in the cell at the specified edge index, then insert it into the isosurface description (IsoMesh.Vertices, IsoMesh.Polygons) so the polygon mesh can be constructed.  In other words, same as in Marching Tetrahedra, but you'll be reading potentially multiple triangles instead of a single triangle each time a lookup is performed.

As with Marching Tetrahedra, the polygons should be defined so the normals point towards the positive vertices of the cell.

In addition to aCaseMap, define the __aIndicesSource0__ and __aIndicesTarget0__ lists to define the vertex pairings which define the cell's edges.

__TIP__: Consult Wenger's diagrams for the possible cube configurations.

__TIP__: Consult the companion lab "__Marching_Cubes_Table.ipynb__" included with this project.  This jupyter notebook will walk you through the process of constructing a lookup table using an algorithmic approach to automate the process, and may prove faster and less error prone than defining the table manually.

In [ ]:
aIndicesSource0 = [0,1,2,3, 4,5,6,7, 0,1,2,3]   # edge connection order in the cell
aIndicesTarget0 = [1,2,3,0, 5,6,7,4, 4,5,6,7]

### lookup table for Marching Cubes ###

# each index of the array contains polygon connectivity information.
# The first value is N, the number of sides in the polygon.
# The trailing values are the indices of the edges within the cube
# and the order they must be connected to form a polygon.
# It is the developer's responsibility to map the edge indices
# to vertex indices of the vertices on those edges so meshing can take place.
#
# NOTE: some cells contain multiple polygons and must be parsed from the returned array index.

aCaseMap = [
    [3,0,1,2],    # <-- case 0: not used
    .
    .
    .
    [3,4,5,6]     # <-- case 255: not used
]

# Part 7: Update classes

## Modify Cell

Once again, the Cell class must be updated to account for Marching Cubes.  Copy and paste your Cell class into the cell below, then make the following modifications:

- Read the `nbTetra` parameter from the Marching Tetrahedra implementation and implement support for Marching Cubes for cases where `nbTetra` is a value of 0.

In [ ]:
### copy and paste your 'Cell' class here, then make the necessary modifications.

## Modify IsoMesh

Copy and paste your Isomesh class into the cell below, then make the following modifications:

- __createIsoSurface()__: When nbTetra = 0, use Marching Cubes. 
- __evalCell()__: When nbTetra = 0, use Marching Cubes lookup tables, and bypass evalTetra().  Must read and parse the triangle indices returned by the lookup tables for cases where multiple triangles are drawn in the cell.

In [ ]:
### copy and paste your 'IsoMesh' class here, then make the necessary modifications.

# Part 8: Define the SuperQuadric shape and create the IsoSurface

Now it's time to play.  Define the superquadric parameters, create the scalar field, then generate the isosurface using the 3 Marching algorithms we implemented.  Store each polygon mesh separately so we may display and compare them in a plot.

The default scalar field should span from [-5,-5,-5] to [5,5,5] with cell size of 0.25 units, but you are free to define any size you like.  Keep in mind execution time is proportional to the number of cells in the scalar field.  So pay attention to the cell size in relation to the scalar field dimensions.  If you're not careful, you can easily exceed system memory leading to extreme performance degradation or crash.

Some helpful values for the superquadric's exponents parameter:
- __(0...1)__: Concave edge.  Forms a star-like shape when applied to all axes.
- __1.0__: Linear edge.  Forms an octahedron when applied to all axes.
- __(1...2)__: Convex edge.  Forms an ellipsoid when applied to all axes.
- __2.0__: Sphere (ellipsoid).  Forms a sphere when applied to all axes.
- __(2...inf)__: Cuboid with rounded corners.  Higher exponents creates sharper corners.

In [ ]:
aMethods = [0,5,6]         # isosurfacing algorithms (0=Marching cubes, 5=Marching Tetrahedra 5, 6=Marching tetrahedra 6)
aPolygonMeshes = []
aTimes = []

# superquadric parameters
sq_scale     = [4,4,4]          # scaling factor of local axes of the superqudric primitive
sq_exponents = [4,4,4]    # exponents defning the primitive's shape (see above)

# scalar field parameters
cellSize   = 0.25               # spacing between adjacent grid points along an axis
bbmin      = [-5,-5,-5]         # min corner of bounding box
bbmax      = [5,5,5]            # max corner of bounding box
dimensions = np.subtract( bbmax, bbmin )
isovalue   = 1.0

sw = StopWatch(0)

# generate the scalar field
field = createScalarField( cellSize, bbmin, bbmax, sq_scale, sq_exponents )
print( "create scalar field: ", sw.GetSplit(), "seconds" )

# create an isomesh instance
isoMesh = IsoMesh( field.dimension[X] - 1, field.dimension[Y] - 1, field.dimension[Z] - 1 )

for i in range(0,3):
    
    print( "----------------------" )
    
    # reset for next isosurface
    isoMesh.clear()
    isoMesh.TetraCount = aMethods[i]
    
    # create isosurface at isovalue
    polygonMesh = isoMesh.createIsoSurface( field, isovalue )
    if polygonMesh:
        # record the isosurface mesh
        aPolygonMeshes.append( polygonMesh )
        
    # record total time for this mesh
    print( "isoMesh[",i,"]", sw.GetSplit() )
    aTimes.append( sw.GetSplit() )

# Part 9: Display the IsoSurfaces

In this section we'll use PyVista.Plotter().  It can accommodate multiple polygon meshes and is more practical than PolyData.plot() as we can add text and other decoration more easily to enhance the result.

Each iteration through the loop will generate a new plotter which in turn has it's own camera, polygon mesh, and titles.

In [ ]:
titles = ["Marching Cubes", "Marching Tetrahedra 5", "Marching Tetrahedra 6"]

plotter = pv.Plotter( shape=(1, len( aPolygonMeshes ) ) )

for i in range( 0, len( aPolygonMeshes ) ):
                     
    mesh = aPolygonMeshes[i]
    
    # create a new subplot to occupy part of the final frame.
    plotter.subplot(0,i)
    
    # set camera
    plotter.camera_position = [
        (10,25,7),   # position
        (0,0,1),     # lookat
        (0,0,1)      # up vector
    ]
    
    # apply text
    plotter.add_text( titles[i] +
        '\n cell size: \t' + str(cellSize) + 
        '\ndimensions: \t' + str(dimensions) +
        '\naxes scale: \t' + str(sq_scale) + 
        '\n exponents: \t' + str(sq_exponents) +
        '\n  Vertices: \t' + str( len( mesh.points ) ) +
        '\n  Polygons: \t' + str( mesh.n_faces ),
        font_size=12,
        font='courier'
    )
    
    # add the mesh with colormap
    plotter.add_mesh( mesh, show_edges=True, scalars=mesh.points[:,Z], stitle='Z coordinate'  )
    
    # add scalar bar
#     plotter.add_scalar_bar( "Z" )

# display the result
plotter.show(  window_size=[1920,720], screenshot="final_results.png" )

# Part 10: Analysis
As we can see from the collected data, Marching Cubes performs better than either of the Marching Tetrahedra algorithms producing isosufaces with fewer polygons and vertices in less time.  The Marching Cubes mesh also look more aesthetically pleasing.

__TBD__

# Part 11: Improvements
    
Congratulations, you have completed the project!  While you should be proud of your accomplishment, be aware you have only completed a basic implementation of the Marching algorithms.  Adequate as a learning exercise, but not ideal for practice.  Consider the following:

1. The Marching Cubes algorithm produces triangles in the resulting polygon mesh. If we could merge triangles to form quadrilateral polygons in cases where the triangles are expected to be coplanar, the vertex and polygon counts could be significantly reduced saving valuable computational resources and lead to a much more pleasing result.  How much reduction can be achieved?  How would you implement this feature?  


2. The cells are accumulated as the scalar field is traversed, but we never backtrack more than an adjacent cell neighbor.  This means we store cells much longer than necessary leading to excessive memory consumption, and potentially slowing execution.  How would you fix that?


3. Our algorithm works in a classic single-threaded model.  Could we accelerate the algorithm using parallel computing?  Why or why not?


4. The Marching Tetrahedra algorithms avoid ambiguity, but introduce an excessive number of triangles that do not necessarily improve the shape of the resulting isosurface.  Why do the extra triangles not improve the isosurface?  How can this be improved?


5. Inspection reveals most of the time is spent copying values onto cell vertices whether it be from the scalar field or from adjacent cells.  How would you modify the algorithm to be more efficient in this regard?


6. What other improvements do you see that could be made?